In [30]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import nltk
import numpy as np
import nltk

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [ ]:
# ['gameplay', 'price', 'cheat', 'optimization', 'graphic']

In [3]:
from ast import literal_eval
df_test = pd.read_csv('feature_extraction_pubg.csv',sep = '\t')
# if need to transform pandas series back to original type
position_list = [literal_eval(dic) for dic in df_test['feature_positions']]
print(position_list[0])

{1: 0, 3: 4}


In [17]:
df_test.loc[9999,]['review']

'its ok but need high power PC for good performance'

In [20]:
r = df_test.loc[9999,]['review']
words = nltk.word_tokenize(r)
words[9]

'performance'

In [19]:
df_test.loc[9999,]

review               its ok but need high power PC for good perform...
feature_positions                                               {9: 3}
Name: 9999, dtype: object

In [65]:
W = 5
position_list = [literal_eval(dic) for dic in df_test['feature_positions']]
reviews = df_test['review'].tolist()
review_windows = dict()

In [74]:
for review_idx in range(len(reviews)):
    if review_idx % 10000 == 0:
        print(review_idx)
    review = reviews[review_idx]
    tokens = nltk.word_tokenize(review)
    feature_pos = position_list[review_idx]
    sentiment_windows = []
    for token_index, category in feature_pos.items():
        if token_index <= W:
            candidate = " ".join(tokens[:token_index+W+1])
        elif token_index+W > len(tokens):
            candidate = " ".join(tokens[token_index-W:])
        else:
            candidate = " ".join(tokens[token_index-W:token_index+W+1])
        sentiment_windows.append((candidate,category))
    review_windows[review_idx] = sentiment_windows

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [21]:
# n_reviews = len(df_test)
# for i in range(n_reviews):
#     review = df_test.loc[i,]['review']
#     feature_positions = df_test.loc[i,]['feature_positions']

129539

## Sentiment Analysis Model 1

In [31]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [33]:
# nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jaclyn/nltk_data...


True

In [35]:
sia = SIA()
results = []

In [39]:
for line in reviews[:10]:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

pprint(results[1], width=100)

{'compound': -0.0,
 'headline': 'Graphics are pretty bad but the gameplay is okay.  The guns are pretty cool and I '
             "like the attachments but it's really boring, and it's annoying as hell when you've "
             'been in a match for half an hour and you get killed by a camper with a Mini14. I '
             "would only really recommend this if you don't mind slow-paced games, if you want an "
             'action packed BR play Apex or something.\r'
             '\r\n'
             '6/10',
 'neg': 0.217,
 'neu': 0.526,
 'pos': 0.257}


In [47]:
pol_score = sia.polarity_scores(reviews[9999])
pol_score['headline'] = reviews[9999]
pol_score

{'neg': 0.0,
 'neu': 0.595,
 'pos': 0.405,
 'compound': 0.6652,
 'headline': 'its ok but need high power PC for good performance'}

In [83]:
sentiment_results = dict()

for review_id in range(len(reviews)):
    if review_id % 10000 == 0:
        print(review_id)
    sent_cat = list()
    review = reviews[review_id]
    position = position_list[review_id]
    review_window = review_windows[review_id]
    for sent_tuple in review_window:
        line = sent_tuple[0]
        category = sent_tuple[1]
        try:
            sent = sia.polarity_scores(line)
        except IndexError:
            print(IndexError)
            continue
        sent_cat.append((category, sent))
    sentiment_results[review_id] = sent_cat

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [84]:
sentiment_results

{0: [(0, {'neg': 0.438, 'neu': 0.562, 'pos': 0.0, 'compound': -0.5994}),
  (4, {'neg': 0.358, 'neu': 0.642, 'pos': 0.0, 'compound': -0.5994})],
 1: [(0, {'neg': 0.164, 'neu': 0.511, 'pos': 0.325, 'compound': 0.296}),
  (0, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})],
 2: [(2, {'neg': 0.393, 'neu': 0.607, 'pos': 0.0, 'compound': -0.7037}),
  (0, {'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'compound': -0.5898}),
  (1, {'neg': 0.231, 'neu': 0.769, 'pos': 0.0, 'compound': -0.4588}),
  (1, {'neg': 0.34, 'neu': 0.494, 'pos': 0.167, 'compound': -0.4215})],
 3: [(0, {'neg': 0.254, 'neu': 0.175, 'pos': 0.57, 'compound': 0.5574}),
  (1, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
  (1, {'neg': 0.0, 'neu': 0.794, 'pos': 0.206, 'compound': 0.0772}),
  (1, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})],
 4: [(0, {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.8481})],
 5: [(0, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})],
 6: [(3, {'neg': 0.0, 'neu': 

In [113]:
cat_0_list = list()
cat_1_list = list()
cat_2_list = list()
cat_3_list = list()
cat_4_list = list()

for review_id in range(len(reviews)):
    cat_list = [0,0,0,0,0]
    n_list = [0,0,0,0,0]

    for sent in sentiment_results[review_id]:
        category = sent[0]
        score = sent[1]['compound']
        cat_list[category] += score
        n_list[category] += 1

    score_list = [0,0,0,0,0]
    for i in range(5):
        if n_list[i] != 0:
            score_list[i] = cat_list[i] / n_list[i]

    cat_0_list.append(score_list[0])
    cat_1_list.append(score_list[1])
    cat_2_list.append(score_list[2])
    cat_3_list.append(score_list[3])
    cat_4_list.append(score_list[4])

In [131]:
all_list = [cat_0_list, cat_1_list, cat_2_list, cat_3_list, cat_4_list]

In [137]:
from sklearn import preprocessing
minmax = preprocessing.MinMaxScaler()
cat_0_minmax = minmax.fit_transform(np.array(cat_0_list).reshape(-1,1)).flatten()
cat_1_minmax = minmax.fit_transform(np.array(cat_1_list).reshape(-1,1)).flatten()
cat_2_minmax = minmax.fit_transform(np.array(cat_2_list).reshape(-1,1)).flatten()
cat_3_minmax = minmax.fit_transform(np.array(cat_3_list).reshape(-1,1)).flatten()
cat_4_minmax = minmax.fit_transform(np.array(cat_4_list).reshape(-1,1)).flatten()

In [138]:
# ['gameplay', 'price', 'cheat', 'optimization', 'graphic']
df_dict = {
    'review': reviews,
    'gameplay': cat_0_minmax,
    'price': cat_1_minmax,
    'cheat': cat_2_minmax,
    'optimization': cat_3_minmax,
    'graphic': cat_4_minmax,
}

df = pd.DataFrame(df_dict)

In [143]:
df

,review,gameplay,price,cheat,optimization,graphic
0,CHUTE LOOT DIE THATS PUBG IN A NUT SHELL,0.187646,0.503536,0.506471,0.506423,0.191143
1,Graphics are pretty bad but the gameplay is ok...,0.574639,0.503536,0.506471,0.506423,0.504522
2,They will ruin your reputation with false bans...,0.192616,0.272958,0.139234,0.506423,0.504522
3,☼ Graphics ☼\r\r\n☐ Bad\r\r\n☐ Alright\r\r\n☐ ...,0.786620,0.517017,0.506471,0.506423,0.504522
4,Excellent game to play with friends,0.937141,0.503536,0.506471,0.506423,0.504522
...,...,...,...,...,...,...
129534,"I only played maybe an hour of this game, then...",0.498007,0.503536,0.394870,0.506423,0.504522
129535,Personally I think its better than warzone jus...,0.726039,0.503536,0.506471,0.506423,0.504522
129536,"Honestly this was the best battle royal ever, ...",0.498007,0.503536,0.506471,0.506423,0.704136
129537,games full of hackers and exploiters,0.498007,0.503536,0.506471,0.506423,0.504522


In [140]:
df.to_csv("sentiment_feature.csv")

## Sentiment Analysis Model 2

In [144]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

KeyboardInterrupt: 